# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets.

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.

Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:*

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [4]:
# your code here
import pandas as pd

# Load the dataset (assuming the file is available at the given path)
file_path = 'training.1600000.processed.noemoticon.csv'

# Use on_bad_lines='skip' instead of error_bad_lines=False for older pandas versions
df = pd.read_csv(file_path, encoding='latin-1', header=None, on_bad_lines='skip')

# Define the column names
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']

# Display basic information about the dataset
print(df.info())

# Take a sample of 20,000 records to work with initially
sample_df = df.sample(n=20000, random_state=42)

# Display the first few rows of the sample
print(sample_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB
None
        target          id                          date      flag  \
541200       0  2200003196  Tue Jun 16 18:18:12 PDT 2009  NO_QUERY   
750          0  1467998485  Mon Apr 06 23:11:14 PDT 2009  NO_QUERY   
766711       0  2300048954  Tue Jun 23 13:40:11 PDT 2009  NO_QUERY   
285055       0  1993474027  Mon Jun 01 10:26:07 PDT 2009  NO_QUERY   
705995       0  2256550904  Sat Jun 20 12:56:51 PDT 2009  NO_QUERY   

                   user                                               text  
541200  LaLaLindsey0609       

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
import re

def clean_up(text):
    # Step 1: Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Step 2: Replace special characters and numbers with whitespace
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # Step 3: Convert to lowercase
    text = text.lower()

    # Step 4: Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [6]:
def tokenize(text):
    # Split the text into words using whitespace
    tokens = text.split()
    return tokens

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Download WordNet resource
nltk.download('wordnet')

# Initialize the lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def stem_and_lemmatize(words):
    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # Stem each word
    stemmed_words = [stemmer.stem(word) for word in words]

    return lemmatized_words, stemmed_words

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
import nltk
from nltk.corpus import stopwords

# Download stopwords
nltk.download('stopwords')

# Get the list of English stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(words):
    # Remove stopwords from the list
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return filtered_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Apply the functions in sequence to create the 'text_processed' column
def process_text(text):
    # Step 1: Clean up text (remove URLs, special characters, etc.)
    cleaned_text = clean_up(text)

    # Step 2: Tokenize cleaned text
    tokens = tokenize(cleaned_text)

    # Step 3: Stem and Lemmatize tokens (use lemmatized tokens for further analysis)
    lemmatized_tokens, _ = stem_and_lemmatize(tokens)

    # Step 4: Remove stop words (applied on lemmatized tokens)
    final_tokens = remove_stopwords(lemmatized_tokens)

    return final_tokens

# Apply the process_text function to each row in the 'text' column
sample_df['text_processed'] = sample_df['text'].apply(process_text)

# Display the first few rows of the updated DataFrame
print(sample_df[['text', 'text_processed']].head())


                                                     text  \
541200             @chrishasboobs AHHH I HOPE YOUR OK!!!    
750     @misstoriblack cool , i have no tweet apps  fo...   
766711  @TiannaChaos i know  just family drama. its la...   
285055  School email won't open  and I have geography ...   
705995                             upper airways problem    

                                           text_processed  
541200                    [chrishasboobs, ahhh, hope, ok]  
750              [misstoriblack, cool, tweet, apps, razr]  
766711  [tiannachaos, know, family, drama, lame, hey, ...  
285055  [school, email, open, geography, stuff, revise...  
705995                           [upper, airway, problem]  


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [10]:
# your code here
from nltk import FreqDist

# Step 1: Combine all words in text_processed into a single list
all_words = [word for tokens in sample_df['text_processed'] for word in tokens]

# Step 2: Calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

# Step 3: Select the top 5,000 words based on frequency
top_5000_words = [word for word, _ in freq_dist.most_common(5000)]

# Display the top 10 words for verification
print("Top 10 words:", top_5000_words[:10])

# Step 1: Define a list of meaningless words to remove
meaningless_words = {'wa', 'quot', 'amp', 'http', 'rt'}  # Add common meaningless artifacts here

# Step 2: Filter out these meaningless words from the frequency distribution
filtered_freq_dist = {word: freq for word, freq in freq_dist.items() if word not in meaningless_words}

# Step 3: Select the top 5,000 words based on the filtered frequency
filtered_top_5000_words = [word for word, _ in sorted(filtered_freq_dist.items(), key=lambda item: item[1], reverse=True)[:5000]]

# Display the top 10 filtered words for verification
print("Top 10 filtered words:", filtered_top_5000_words[:10])




Top 10 words: ['wa', 'day', 'good', 'get', 'like', 'quot', 'go', 'today', 'work', 'love']
Top 10 filtered words: ['day', 'good', 'get', 'like', 'go', 'today', 'work', 'love', 'u', 'time']


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [11]:
# your code here
from sklearn.feature_extraction.text import CountVectorizer

# Step 1: Define a function to convert lists of words to text (since CountVectorizer works with text input)
def list_to_string(words):
    return ' '.join(words)

# Apply this function to the 'text_processed' column to convert lists of words to strings
sample_df['processed_text_str'] = sample_df['text_processed'].apply(list_to_string)

# Step 2: Create a CountVectorizer instance using the top 5,000 words as the vocabulary
vectorizer = CountVectorizer(vocabulary=filtered_top_5000_words)

# Step 3: Fit and transform the 'processed_text_str' to create the Bag of Words matrix
bow_matrix = vectorizer.fit_transform(sample_df['processed_text_str'])

# Display the Bag of Words matrix shape (number of documents, number of features)
print("Bag of Words Matrix Shape:", bow_matrix.shape)

# Optional: Convert the BoW matrix to a DataFrame for a more readable view
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())
print(bow_df.head())


Bag of Words Matrix Shape: (20000, 5000)
   day  good  get  like  go  today  work  love  u  time  ...  kidnap  sharp  \
0    0     0    0     0   0      0     0     0  0     0  ...       0      0   
1    0     0    0     0   0      0     0     0  0     0  ...       0      0   
2    0     0    0     1   0      0     0     0  0     1  ...       0      0   
3    0     0    0     0   0      0     0     0  0     0  ...       0      0   
4    0     0    0     0   0      0     0     0  0     0  ...       0      0   

   cal  protest  ticklemejoey  bracelet  threat  weekly  cartoon  visitor  
0    0        0             0         0       0       0        0        0  
1    0        0             0         0       0       0        0        0  
2    0        0             0         0       0       0        0        0  
3    0        0             0         0       0       0        0        0  
4    0        0             0         0       0       0        0        0  

[5 rows x 5000 columns]


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [12]:
# your code here
from sklearn.model_selection import train_test_split

# Split the feature set and labels (sample_df['target'] represents sentiment labels)
X = bow_df  # Features (Bag of Words)
y = sample_df['target']  # Labels (sentiments)

# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
import numpy as np

# Convert the training data into feature dictionaries
def to_feature_dict(row):
    return {f"word_{i}": row[i] for i in range(len(row))}

# Prepare training data in the required format
train_data = [(to_feature_dict(X_train.iloc[i].values), y_train.iloc[i]) for i in range(len(X_train))]

# Convert the data labels to binary form expected by nltk (0 or 1)
train_data = [(features, 'positive' if label == 4 else 'negative') for features, label in train_data]


In [14]:
import nltk
from nltk.classify import NaiveBayesClassifier

# Train the Naive Bayes Classifier
classifier = NaiveBayesClassifier.train(train_data)


In [15]:
# Show the most informative features
classifier.show_most_informative_features()


Most Informative Features
                word_239 = 1              negati : positi =     23.3 : 1.0
                word_632 = 1              negati : positi =     18.5 : 1.0
                word_460 = 1              positi : negati =     17.2 : 1.0
                word_677 = 1              negati : positi =     16.5 : 1.0
                word_334 = 1              positi : negati =     16.2 : 1.0
                 word_43 = 1              negati : positi =     14.2 : 1.0
                word_516 = 1              negati : positi =     14.0 : 1.0
                word_707 = 1              negati : positi =     13.8 : 1.0
                word_872 = 1              negati : positi =     13.1 : 1.0
                word_821 = 1              positi : negati =     12.9 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here
# Prepare the test data in the required format
test_data = [(to_feature_dict(X_test.iloc[i].values), y_test.iloc[i]) for i in range(len(X_test))]

# Convert the data labels to binary form expected by nltk (0 or 1)
test_data = [(features, 'positive' if label == 4 else 'negative') for features, label in test_data]
from nltk.classify import accuracy

# Calculate the accuracy of the classifier on the test set
test_accuracy = accuracy(classifier, test_data)

# Print the accuracy score
print("Naive Bayes Classifier Accuracy:", test_accuracy)

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time.

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here